<a href="https://colab.research.google.com/github/lygitdata/STATS-415-Project/blob/main/code/STATS_415_Kaggle_Li_Yuan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# Data Cleaning

In [ ]:
# Import training set
X_train = pd.read_csv('https://stats415.ly.gd.edu.kg/kaggleData/X_train.csv')
y_train = pd.read_csv('https://stats415.ly.gd.edu.kg/kaggleData/y_train.csv')
# Import testing set
X_test = pd.read_csv('https://stats415.ly.gd.edu.kg/kaggleData/X_test.csv')
y_sample = pd.read_csv('https://stats415.ly.gd.edu.kg/kaggleData/y_sample.csv')

# Dummy variable transform
categorical_cols = ['self_eval', 'teacher_eval', 'district']
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
print(X_train.shape)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)
print(X_test.shape)

# Drop and save SEQN
SEQN_X_train = X_train['SEQN']
X_train = X_train.drop(columns=['SEQN'])
SEQN_y_train = y_train['SEQN']
y_train = y_train.drop(columns=['SEQN'])
SEQN_X_test = X_test['SEQN']
X_test = X_test.drop(columns=['SEQN'])

# Convert data to numpy arrays
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()

(8000, 65)
(4000, 65)


# Model Training

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0008,
    decay_steps=2048,
    decay_rate=0.95
)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    ),
    keras.callbacks.ModelCheckpoint(
        "best_model.h5",
        save_best_only=True,
        monitor='val_loss'
    )
]

def build_model():
    model = Sequential([
        layers.Dense(units=4096, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(units=1024, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(units=512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(units=1, activation='linear')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        loss="mean_squared_error",
        metrics=[tf.keras.metrics.R2Score(name='r2_score')]
    )

    return model

kf = KFold(n_splits=10, shuffle=True, random_state=42)

mse_scores = []

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    model = build_model()
    history = model.fit(
        X_train_fold, y_train_fold,
        epochs=200,
        batch_size=512,
        validation_data=(X_val_fold, y_val_fold),
        callbacks=callbacks
    )

    y_val_pred = model.predict(X_val_fold)
    mse = mean_squared_error(y_val_fold, y_val_pred)
    mse_scores.append(mse)

print("Mean MSE:", np.mean(mse_scores))
print("Std Dev MSE:", np.std(mse_scores))

Epoch 1/200
15/15 [==============================] - 5s 46ms/step - loss: 1.7910 - r2_score: 0.2308 - val_loss: 1.0338 - val_r2_score: 0.5207
Epoch 2/200
 1/15 [=>............................] - ETA: 0s - loss: 1.0662 - r2_score: 0.5617

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15/15 [==============================] - 0s 23ms/step - loss: 1.0682 - r2_score: 0.5412 - val_loss: 0.8795 - val_r2_score: 0.5923
Epoch 3/200
15/15 [==============================] - 0s 22ms/step - loss: 0.8812 - r2_score: 0.6216 - val_loss: 0.8060 - val_r2_score: 0.6263
Epoch 4/200
15/15 [==============================] - 0s 11ms/step - loss: 0.8209 - r2_score: 0.6475 - val_loss: 0.8102 - val_r2_score: 0.6244
Epoch 5/200
15/15 [==============================] - 0s 23ms/step - loss: 0.7355 - r2_score: 0.6841 - val_loss: 0.7442 - val_r2_score: 0.6550
Epoch 6/200
15/15 [==============================] - 4s 296ms/step - loss: 0.6700 - r2_score: 0.7123 - val_loss: 0.7083 - val_r2_score: 0.6716
Epoch 7/200
15/15 [==============================] - 1s 62ms/step - loss: 0.6193 - r2_score: 0.7340 - val_loss: 0.6578 - val_r2_score: 0.6950
Epoch 8/200
15/15 [==============================] - 0s 28ms/step - loss: 0.5664 - r2_score: 0.7567 - val_loss: 0.6128 - val_r2_score: 0.7159
Epoch 9/200
15/15

In [ ]:
y_temp = model.predict(X_train)
mean_squared_error(y_temp, y_train)

250/250 [==============================] - 0s 2ms/step


0.09919011099189

In [ ]:
y_test = model.predict(X_test)

125/125 [==============================] - 0s 3ms/step


In [ ]:
output = pd.DataFrame({'SEQN': SEQN_X_test.values.flatten(), 'y': y_test.flatten()})
output

,SEQN,y
0,492834,0.694200
1,309349,-1.235504
2,468308,-1.248916
3,838812,-2.810004
4,947936,0.609807
...,...,...
3995,971604,0.245420
3996,2790,-0.128860
3997,159210,-0.531284
3998,366040,0.261004


In [ ]:
out = y_sample.merge(output, on=['SEQN'], how="left")
out

,SEQN,y_x,y_y
0,492834,-0.664754,0.694200
1,309349,-1.212264,-1.235504
2,468308,0.845806,-1.248916
3,838812,-1.488121,-2.810004
4,947936,-0.632632,0.609807
...,...,...,...
3995,971604,1.287817,0.245420
3996,2790,1.213925,-0.128860
3997,159210,2.221951,-0.531284
3998,366040,-1.858236,0.261004


In [ ]:
out = out.drop(columns=['y_x'])
out = out.rename(columns={'y_y': 'y'})
out

,SEQN,y
0,492834,0.694200
1,309349,-1.235504
2,468308,-1.248916
3,838812,-2.810004
4,947936,0.609807
...,...,...
3995,971604,0.245420
3996,2790,-0.128860
3997,159210,-0.531284
3998,366040,0.261004


In [ ]:
out.to_csv('result.csv', index=False)